In [1]:
%cd ./drive/MyDrive/heart_segmentation_ENMC/

/content/drive/MyDrive/heart_segmentation_ENMC


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
from unet import unet_reduzida, unet_completa, unet_n, dice_coef
from utils import get_images, separate_train_test, make_vol, image_preprocess
import glob
from keras import backend as K
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from time import time

hd = [45 , 25, 40 , 40 , 13, 40, 42 , 33, 6 , 43 , 37 , 8 , 28, 18, 46 , 37, 41 , 41 , 33, 33 ]
hu = [112, 95, 113, 116, 91, 98, 109, 98, 67, 105, 104, 81, 97, 88, 103, 90, 110, 113, 92, 101]

x_train = sorted(glob.glob('data_heart/imagesTr/*'))
y_train = sorted(glob.glob('data_heart/labelsTr/*'))
img_xtrain, index_x = get_images(x_train,hd,hu)
img_ytrain, index_y = get_images(y_train,hd,hu)

#predict_vol = []
dice_metric = []
tempo = []
local = 'heart_tests/heart_LOO_ES/'


Using TensorFlow backend.


In [ ]:
# nº 2 DataAug


data_gen_args = dict(shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True)

image_datagen = ImageDataGenerator(**data_gen_args)


image_generator = image_datagen.flow(train_X, train_ground,
        batch_size=use_batch_size,
        seed=SEED_2)

epoch = 100
spe = 300

history = model.fit_generator(image_generator, steps_per_epoch=spe, epochs=epoch, callbacks=callback)

In [ ]:
# nº 3 FMédias

def mean_filter(image, filter_size):
    new_image = cv2.blur(image,(filter_size, filter_size))
    return new_image
def apply_mean_filter():
    filter_size = 9 
    imgs_list = []
    for img in imgs_load:
        imgs_list.append(mean_filter(img, filter_size))
    return imgs_list

In [ ]:
train_X.shape

In [ ]:
for i in range(len(img_xtrain)):
    
    print("Rodando Pela %i vez"%(i+1))
    
    train_X, valid_X, index_train_x, index_test_x = separate_train_test(img_xtrain, index = i)
    train_ground, valid_ground, index_train_y, index_test_y = separate_train_test(img_ytrain, index = i)
    
    size_img = 128
    
    train_X, valid_X, train_ground, valid_ground = image_preprocess(train_X, valid_X, train_ground, valid_ground)
    
    inicial = time()
    model = unet_n(size_img, mult = 2)
    
    reduce = ReduceLROnPlateau(monitor = 'loss', patience = 2, verbose = 1)
    es = EarlyStopping(monitor = 'loss', patience = 5, verbose = 1)
    
    history = model.fit(x = train_X, y = train_ground, batch_size = 4, epochs = 100, callbacks = [reduce, es])
    final = time()
    
    model.save(local + 'Heart_Model_%i.h5'%(i))
        
    predicao = model.predict(valid_X)
    #predict_vol.append(make_vol(predicao, index = index_test_y))
    predicao = predicao > 0.5
    predicao = np.float64(predicao)

    sess = tf.InteractiveSession()
    dice_metric.append(dice_coef(predicao, valid_ground).eval())
    sess.close()

    tempo_total = final - inicial
    tempo.append(tempo_total)
    print("rodou em : %f"%(tempo_total))
    K.clear_session()


In [ ]:
np.savetxt(local + 'Dice Metric.txt', dice_metric)
np.savetxt(local + 'Tempo.txt', tempo)